In [3]:
#basic libraries
import pandas as pd
import numpy as np
import itertools
from itertools import chain
import json

#nlp Libraries
from odenet import *
import spacy
from spacy.lang.de.examples import sentences 
nlp = spacy.load("de_core_news_lg")


In [7]:
synonyms_word("platzierung")

None


In [8]:
#create new intents as dictionary
data = dict()
data['Soccer_Intents'] = []


In [16]:
for i in range(len(data['Soccer_Intents'])):
        
    intent = data['Soccer_Intents'][i]
    tag = intent['tag']
    pattern = intent['patterns']

    
    #single words like headache
    
    #get synonyms for the tag
    synonyms = wordnet.synsets(tag)
    new_words = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
    new_words = [word.replace('_', ' ') for word in new_words] #this is a list with every synonym found by the wordnet from nltk
    
    docs = []
    for item in new_words:
        
        #filter out synonyms that have no scientific background
        #note that this step was due to weird synonyms like "make out" as a synonym for "neck" (the body part)
        #we only could use a scientific dataset from sci-spacy but would have favored a medical dataset here. (More on that at the end)
        doc = list(nlp(item).ents)

        X = []
        for x in doc:
            x = str(x)
            X.append(x)

        X = ' '.join(X)
        docs.append(X)
    
    
    
    pattern.extend(docs)
    
    #two words like neck_pain
    #same process es above but for each of the two words we find synonyms each, then filter them and then combine them to one pattern.
    sentence = tag.split('_')
    if len(sentence) == 2:
        liste=[]
        for part in sentence:
            synonyms = wordnet.synsets(part)
            part = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
            part = [word.replace('_', ' ') for word in part]

            docs= []
            for item in part:
                doc = list(nlp(item).ents)

                X = []
                for x in doc:
                    x = str(x)

                    X.append(x)
                    
                X = ' '.join(X)
                docs.append(X)

            liste.append(docs)

        liste2=[]
        for part in liste:
            part = [x for x in part if x]
            liste2.append(part)

        liste2 = list(itertools.product(liste2[0], liste2[1]))
        
        for sentence in liste2:
            string = ' '.join(sentence) 

            pattern.append(string)
    

In [18]:
pattern

NameError: name 'pattern' is not defined

In [14]:
    #add pattern and the belonging tag/intent/synonym to the intents

pattern = list(set(pattern))
    
dictionary = dict()
dictionary['tag']=tag
dictionary['patterns']=pattern
    
data['intents'].append(dictionary)

NameError: name 'pattern' is not defined

In [17]:
#save intents as new intents
json_object = json.dumps(data, indent = 4)

with open("intents_new2.json", "w") as outfile:
    outfile.write(json_object)